In [1]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD, evaluate
%matplotlib inline

In [2]:
df = pd.read_csv("/Users/nicholastang/Desktop/rating.csv")

In [3]:
df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [4]:
df_name = pd.read_csv('https://storage.googleapis.com/kaggle-datasets/571/1094/anime.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1519690840&Signature=J%2Fu4uLowUuhS9wp2TimzBzGudwbJP69SvQH5rY9Ap0GaNI9iGUz5%2Fj2BK%2BGNVBO1mYKBpUoZQrbelmz45VgEvvDBJ8NciwIkWUGkLZaSkpYidrv5GIjzzwxaAeWxF%2BycVU45IEdmc%2FdjLt25vEOlQ0W03Iq3oHm9zICRJGlMkwqkdSL7EN9FlLMqONbk%2B8R8ySZLGrnB7bRecucNCAx68giRsyqwc7uds9hbKR%2BZyJSBPXXpvuUtWFaLu1AEG0%2FLYY6CIfuaQOO40u%2FOMtpjAVNtTQyXpC6F33wMIYxybSddOJtGqY0JxMY25qnLy3gHoma0qSaQ9%2FUg4yQrxZE0BQ%3D%3D')
df_name.set_index('anime_id', inplace = True)
df_name = df_name['name']
df_name.head()

anime_id
32281                      Kimi no Na wa.
5114     Fullmetal Alchemist: Brotherhood
28977                            Gintama°
9253                          Steins;Gate
9969                        Gintama&#039;
Name: name, dtype: object

In [5]:
reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'rating']][:100000], reader)
data.split(n_folds=3)

svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

/Users/nicholastang/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/nicholastang/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 3.3028
MAE:  2.9855
------------
Fold 2
RMSE: 3.3182
MAE:  3.0053
------------
Fold 3
RMSE: 3.3096
MAE:  2.9955
------------
------------
Mean RMSE: 3.3102
Mean MAE : 2.9954
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [2.9854538014250243,
                             3.0052643899541938,
                             2.9955460362526276],
                            'rmse': [3.3027820272475825,
                             3.3181854745093053,
                             3.3095717722483884]})

In [6]:
df_73515 = df[(df['user_id'] == 73515) & (df['rating'] == 10)]
df_73515 = df_73515.set_index('anime_id')
df_73515 = df_73515.join(df_name)['name']
df_73515

anime_id
1                                          Cowboy Bebop
5                       Cowboy Bebop: Tengoku no Tobira
6                                                Trigun
33                                              Berserk
72                            Full Metal Panic? Fumoffu
96                              Mobile Fighter G Gundam
166                                           Samurai 7
202                                    Wolf&#039;s Rain
205                                    Samurai Champloo
239                                          Gankutsuou
240                                           Genshiken
617                                   Juubee Ninpuuchou
777                                   Hellsing Ultimate
1535                                         Death Note
1575                    Code Geass: Hangyaku no Lelouch
1587                       Tenpou Ibun: Ayakashi Ayashi
1813                                      Genshiken OVA
1860                   Tokyo Majin Gaku

In [7]:
user_73515 = df_name.copy()
user_73515 = user_73515.reset_index()

# getting full dataset
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'rating']][:100000], reader)

trainset = data.build_full_trainset()
svd.train(trainset)

user_73515['Estimate_Score'] = user_73515['anime_id'].apply(lambda x: svd.predict(73515, x).est)

user_73515 = user_73515.drop('anime_id', axis = 1)

user_73515 = user_73515.sort_values('Estimate_Score', ascending=False)
user_73515.head(10)

/Users/nicholastang/anaconda3/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


,name,Estimate_Score
0,Kimi no Na wa.,5.0
8201,Baku Tech! Bakugan Gachi: Tokubetsu-hen,5.0
8192,Backstage Idol Story,5.0
8193,Bad Badtz-Maru no Ari to Kirigirisu,5.0
8194,Bad Badtz-Maru no Ookami ga Kita!,5.0
8195,Bad Badtz-Maru no Ore no Pochi wa Sekaiichi,5.0
8196,Bad Badtz-Maru no Ore wa Yuutousei,5.0
8197,Bad Badtz-Maru no Otoko Dokyou no Omoiyari,5.0
8198,Baka Mukashibanashi Movie: Jijii Wars,5.0
8199,Bakabon Osomatsu no Karee wo Tazunete Sansenri,5.0
